In [1]:
!pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 1.8/72.0 MB 20.2 MB/s eta 0:00:04
   --- ------------------------------------ 5.8/72.0 MB 18.5 MB/s eta 0:00:04
   ------ --------------------------------- 11.0/72.0 MB 21.5 MB/s eta 0:00:03
   ------- -------------------------------- 13.4/72.0 MB 21.0 MB/s eta 0:00:03
   ---------- ----------------------------- 19.1/72.0 MB 23.7 MB/s eta 0:00:03
   -------------- ------------------------- 26.7/72.0 MB 22.9 MB/s eta 0:00:02
   ----------------- ---------------------- 31.5/72.0 MB 22.7 MB/s eta 0:00:02
   -------------------- ------------------- 36.4/72.0 MB 22.7 MB/s eta 0:00:02
   -------------------- ------------------- 37.5/72.0 MB 22.1 MB/s eta 0:00:02
   ----------------------- ---------------- 43.0/72.0 MB 21.2 MB/s eta 0:00:02
   -------------------------- ------------- 46.9/72.0 MB 21.5 MB/s eta 0:00:02
   --------------------------- ------------ 49.8/72.0 MB 20.6 M

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, roc_curve, auc, 
                             roc_auc_score, classification_report)
import warnings
warnings.filterwarnings('ignore')

In [3]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

In [4]:
# Загрузка датасета
df = pd.read_csv('diabetes.csv')
X = df.drop('Outcome', axis=1)
y = df['Outcome'] # целевая переменная: диабет присутствует или отсутствует
features = X.columns.tolist()

print(f'Размер датасета: {df.shape[0]} x {df.shape[1]}')
print(df.head())

Размер датасета: 768 x 9
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [5]:
# 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'Train: {X_train.shape}')
print(f'Test: {X_test.shape}')

Train: (614, 8)
Test: (154, 8)


- - - - - - - -
**Задание №1:**

Решить задачу классификации  больных методом **случайного леса**.

-	Провевести исследование качества модели от глубины используемых деревьев. Отрисовать зависимость на графике.
-	Провести исследование качества модели от количества подаваемых на дерево признаков. Отрисовать зависимость на графике.
-	Провести исследование качества модели от числа деревьев. Отрисовать на графике, дополнить график данными о времени обучения.


In [6]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Модель"):
    
    start_time = time.time()
    model.fit(X_train, y_train) # обучение
    train_time = time.time() - start_time
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)
    if hasattr(model, "predict_proba"):
        y_test_proba = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_test_proba)
    else:
        y_test_proba = None
        roc_auc = None
    metrics = {
        'Модель': model_name,
        'Время обучения (с)': train_time,
        'Accuracy (train)': accuracy_score(y_train, y_train_predict),
        'Accuracy (test)': accuracy_score(y_test, y_test_predict),
        'Precision': precision_score(y_test, y_test_predict),
        'Recall': recall_score(y_test, y_test_predict),
        'F1-Score': f1_score(y_test, y_test_predict),
        'ROC-AUC': roc_auc
    }
    
    return metrics, y_test_proba

In [8]:
max_depths = [1, 2, 3, 5, 7, 10, 15, 20, 25, 30, None]
depth_results = []
depth_times = []

for depth in max_depths:
    depth_label = depth if depth is not None else "Без ограничений"
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=depth,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    )
    start_time = time.time()
    cv_scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)
    cv_time = time.time() - start_time

    rf.fit(X_train, y_train) # обучение
    y_test_proba = rf.predict_proba(X_test)[:, 1]
    test_roc_auc = roc_auc_score(y_test, y_test_proba)
    depth_results.append({
        'max_depth': depth_label,
        'CV ROC-AUC mean': cv_scores.mean(),
        'CV ROC-AUC std': cv_scores.std(),
        'Test ROC-AUC': test_roc_auc,
        'Время CV (с)': cv_time
    })
    
    print(f'Глубина {depth_label}: ROC - AUC = {cv_scores.mean():.4f} +- {cv_scores.std():.4f}')

depth_df = pd.DataFrame(depth_results)

Глубина 1: ROC - AUC = 0.8225 +- 0.0209
Глубина 2: ROC - AUC = 0.8279 +- 0.0212
Глубина 3: ROC - AUC = 0.8286 +- 0.0217
Глубина 5: ROC - AUC = 0.8360 +- 0.0170
Глубина 7: ROC - AUC = 0.8296 +- 0.0154
Глубина 10: ROC - AUC = 0.8208 +- 0.0181
Глубина 15: ROC - AUC = 0.8198 +- 0.0161
Глубина 20: ROC - AUC = 0.8195 +- 0.0174
Глубина 25: ROC - AUC = 0.8196 +- 0.0178
Глубина 30: ROC - AUC = 0.8196 +- 0.0178
Глубина Без ограничений: ROC - AUC = 0.8196 +- 0.0178


In [ ]:
# Построение графика зависимости ROC-AUC от глубины
plt.figure(figsize=(14, 8))

# График среднего ROC-AUC
plt.subplot(2, 2, 1)
x_labels = [str(d) for d in depth_df['max_depth']]
x_pos = np.arange(len(x_labels))

plt.plot(x_pos, depth_df['CV ROC-AUC mean'], 'o-', linewidth=2, markersize=8, 
         label='Средний ROC-AUC (CV)', color='blue')
plt.fill_between(x_pos, 
                 depth_df['CV ROC-AUC mean'] - depth_df['CV ROC-AUC std'],
                 depth_df['CV ROC-AUC mean'] + depth_df['CV ROC-AUC std'],
                 alpha=0.2, color='blue')

# Нахождение оптимальной глубины
optimal_depth_idx = depth_df['CV ROC-AUC mean'].idxmax()
optimal_depth = depth_df.loc[optimal_depth_idx, 'max_depth']
optimal_auc = depth_df.loc[optimal_depth_idx, 'CV ROC-AUC mean']

plt.axvline(x=optimal_depth_idx, color='red', linestyle='--', alpha=0.7,
            label=f'Оптимум: глубина={optimal_depth} (AUC={optimal_auc:.3f})')

plt.xticks(x_pos, x_labels, rotation=45)
plt.xlabel('Максимальная глубина деревьев', fontsize=12)
plt.ylabel('ROC-AUC (кросс-валидация)', fontsize=12)
plt.title('Зависимость ROC-AUC от глубины деревьев в Random Forest', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(0.7, 0.9)

# График времени обучения
plt.subplot(2, 2, 2)
plt.bar(x_pos, depth_df['Время CV (с)'], color='orange', alpha=0.7)
plt.xticks(x_pos, x_labels, rotation=45)
plt.xlabel('Максимальная глубина деревьев', fontsize=12)
plt.ylabel('Время обучения, сек', fontsize=12)
plt.title('Время кросс-валидации в зависимости от глубины', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')

# График стандартного отклонения
plt.subplot(2, 2, 3)
plt.bar(x_pos, depth_df['CV ROC-AUC std'], color='green', alpha=0.7)
plt.xticks(x_pos, x_labels, rotation=45)
plt.xlabel('Максимальная глубина деревьев', fontsize=12)
plt.ylabel('Станд. отклонение ROC-AUC', fontsize=12)
plt.title('Устойчивость модели (меньше = лучше)', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')

# График ROC-AUC на тестовой выборке
plt.subplot(2, 2, 4)
plt.plot(x_pos, depth_df['Test ROC-AUC'], 's-', linewidth=2, markersize=8,
         label='ROC-AUC на тесте', color='purple')
plt.axvline(x=optimal_depth_idx, color='red', linestyle='--', alpha=0.7)
plt.xticks(x_pos, x_labels, rotation=45)
plt.xlabel('Максимальная глубина деревьев', fontsize=12)
plt.ylabel('ROC-AUC (тестовая выборка)', fontsize=12)
plt.title('Качество на тестовой выборке', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim(0.7, 0.9)

plt.tight_layout()
plt.show()

In [ ]:
# Исследование зависимости от количества признаков
max_features_options = ['sqrt', 'log2', 0.1, 0.2, 0.3, 0.5, 0.7, 0.9, None]
max_features_labels = ['sqrt', 'log2', '10%', '20%', '30%', '50%', '70%', '90%', 'all']

features_results = []

for i, max_feat in enumerate(max_features_options):
    feat_label = max_features_labels[i]
    
    # Создание модели
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=optimal_depth if optimal_depth != "Без огр." else None,
        max_features=max_feat,
        random_state=42,
        n_jobs=-1
    )
    
    # Измерение времени обучения
    start_time = time.time()
    
    # Кросс-валидация для оценки качества
    cv_scores = cross_val_score(rf, X_train, y_train, 
                                cv=5, scoring='roc_auc', n_jobs=-1)
    cv_time = time.time() - start_time
    
    # Обучение на всей обучающей выборке для получения финальных метрик
    rf.fit(X_train, y_train)
    y_test_proba = rf.predict_proba(X_test)[:, 1]
    test_roc_auc = roc_auc_score(y_test, y_test_proba)
    
    features_results.append({
        'max_features': feat_label,
        'CV ROC-AUC mean': cv_scores.mean(),
        'CV ROC-AUC std': cv_scores.std(),
        'Test ROC-AUC': test_roc_auc,
        'Время CV (с)': cv_time
    })
    
    print(f"   • Признаков {feat_label:>4}: ROC-AUC = {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Преобразование результатов в DataFrame
features_df = pd.DataFrame(features_results)

# Построение графиков
plt.figure(figsize=(14, 10))

# Основной график зависимости ROC-AUC от количества признаков
plt.subplot(2, 2, 1)
x_pos = np.arange(len(features_df['max_features']))

plt.plot(x_pos, features_df['CV ROC-AUC mean'], 'o-', linewidth=2, markersize=8, 
         label='Средний ROC-AUC (CV)', color='blue')
plt.fill_between(x_pos, 
                 features_df['CV ROC-AUC mean'] - features_df['CV ROC-AUC std'],
                 features_df['CV ROC-AUC mean'] + features_df['CV ROC-AUC std'],
                 alpha=0.2, color='blue')

# Нахождение оптимального количества признаков
optimal_feat_idx = features_df['CV ROC-AUC mean'].idxmax()
optimal_feat = features_df.loc[optimal_feat_idx, 'max_features']
optimal_auc = features_df.loc[optimal_feat_idx, 'CV ROC-AUC mean']

plt.axvline(x=optimal_feat_idx, color='red', linestyle='--', alpha=0.7,
            label=f'Оптимум: {optimal_feat} признаков (AUC={optimal_auc:.3f})')

plt.xticks(x_pos, features_df['max_features'], rotation=45)
plt.xlabel('Количество признаков в каждом разбиении', fontsize=12)
plt.ylabel('ROC-AUC (кросс-валидация)', fontsize=12)
plt.title('Зависимость ROC-AUC от количества признаков в Random Forest', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(0.75, 0.88)

# График времени обучения
plt.subplot(2, 2, 2)
plt.bar(x_pos, features_df['Время CV (с)'], color='orange', alpha=0.7)
plt.xticks(x_pos, features_df['max_features'], rotation=45)
plt.xlabel('Количество признаков в каждом разбиении', fontsize=12)
plt.ylabel('Время обучения, сек', fontsize=12)
plt.title('Время кросс-валидации', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')

# График стандартного отклонения (устойчивость)
plt.subplot(2, 2, 3)
plt.bar(x_pos, features_df['CV ROC-AUC std'], color='green', alpha=0.7)
plt.xticks(x_pos, features_df['max_features'], rotation=45)
plt.xlabel('Количество признаков в каждом разбиении', fontsize=12)
plt.ylabel('Станд. отклонение ROC-AUC', fontsize=12)
plt.title('Устойчивость модели (меньше = лучше)', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')

# График сравнения CV и тестовой выборки
plt.subplot(2, 2, 4)
plt.plot(x_pos, features_df['CV ROC-AUC mean'], 'o-', linewidth=2, markersize=8,
         label='CV ROC-AUC', color='blue')
plt.plot(x_pos, features_df['Test ROC-AUC'], 's-', linewidth=2, markersize=8,
         label='Test ROC-AUC', color='purple')
plt.axvline(x=optimal_feat_idx, color='red', linestyle='--', alpha=0.7)
plt.xticks(x_pos, features_df['max_features'], rotation=45)
plt.xlabel('Количество признаков в каждом разбиении', fontsize=12)
plt.ylabel('ROC-AUC', fontsize=12)
plt.title('Сравнение CV и тестовой выборки', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(0.75, 0.88)

plt.tight_layout()
plt.show()

In [ ]:
# Исследование зависимости от количества деревьев
n_estimators_options = [1, 5, 10, 20, 30, 50, 75, 100, 150, 200, 300, 500]
n_trees_results = []

for n_trees in n_estimators_options:
    # Создание модели
    rf = RandomForestClassifier(
        n_estimators=n_trees,
        max_depth=optimal_depth if optimal_depth != "Без огр." else None,
        max_features=optimal_feat if optimal_feat not in ['sqrt', 'log2', 'all'] 
        else (optimal_feat if optimal_feat == 'sqrt' else 
              'log2' if optimal_feat == 'log2' else None),
        random_state=42,
        n_jobs=-1
    )
    
    # Измерение времени обучения
    start_time = time.time()
    
    # Кросс-валидация для оценки качества
    cv_scores = cross_val_score(rf, X_train, y_train, 
                                cv=5, scoring='roc_auc', n_jobs=-1)
    train_time = time.time() - start_time
    
    # Обучение на всей обучающей выборке для получения финальных метрик
    rf.fit(X_train, y_train)
    y_test_proba = rf.predict_proba(X_test)[:, 1]
    test_roc_auc = roc_auc_score(y_test, y_test_proba)
    
    # Важность признаков (только для полной модели)
    if n_trees == n_estimators_options[-1]:
        feature_importances = rf.feature_importances_
    
    n_trees_results.append({
        'n_estimators': n_trees,
        'CV ROC-AUC mean': cv_scores.mean(),
        'CV ROC-AUC std': cv_scores.std(),
        'Test ROC-AUC': test_roc_auc,
        'Время обучения (с)': train_time
    })
    
    print(f"   • Деревьев {n_trees:>3}: ROC-AUC = {cv_scores.mean():.4f}, время = {train_time:.2f}с")

# Преобразование результатов в DataFrame
trees_df = pd.DataFrame(n_trees_results)

# Построение графиков зависимости от количества деревьев
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# График 1: Зависимость ROC-AUC от количества деревьев
axes[0, 0].plot(trees_df['n_estimators'], trees_df['CV ROC-AUC mean'], 
                'o-', linewidth=2, markersize=8, color='blue', label='CV ROC-AUC')
axes[0, 0].fill_between(trees_df['n_estimators'],
                        trees_df['CV ROC-AUC mean'] - trees_df['CV ROC-AUC std'],
                        trees_df['CV ROC-AUC mean'] + trees_df['CV ROC-AUC std'],
                        alpha=0.2, color='blue')

# Нахождение точки, после которой улучшение менее 0.001
for i in range(1, len(trees_df)):
    improvement = trees_df.loc[i, 'CV ROC-AUC mean'] - trees_df.loc[i-1, 'CV ROC-AUC mean']
    if improvement < 0.001 and i > 5:
        optimal_trees = trees_df.loc[i-1, 'n_estimators']
        optimal_trees_auc = trees_df.loc[i-1, 'CV ROC-AUC mean']
        axes[0, 0].axvline(x=optimal_trees, color='red', linestyle='--', alpha=0.7,
                          label=f'Оптимум: {optimal_trees} деревьев')
        break

axes[0, 0].set_xlabel('Количество деревьев (n_estimators)', fontsize=12)
axes[0, 0].set_ylabel('ROC-AUC (кросс-валидация)', fontsize=12)
axes[0, 0].set_title('Зависимость ROC-AUC от количества деревьев', fontsize=14)
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# График 2: Время обучения в зависимости от количества деревьев
axes[0, 1].plot(trees_df['n_estimators'], trees_df['Время обучения (с)'], 
                's-', linewidth=2, markersize=8, color='orange')
axes[0, 1].set_xlabel('Количество деревьев (n_estimators)', fontsize=12)
axes[0, 1].set_ylabel('Время обучения, сек', fontsize=12)
axes[0, 1].set_title('Время обучения в зависимости от количества деревьев', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

# График 3: ROC-AUC на тестовой выборке
axes[1, 0].plot(trees_df['n_estimators'], trees_df['Test ROC-AUC'], 
                '^-', linewidth=2, markersize=8, color='purple')
if 'optimal_trees' in locals():
    optimal_test_auc = trees_df[trees_df['n_estimators'] == optimal_trees]['Test ROC-AUC'].values[0]
    axes[1, 0].axvline(x=optimal_trees, color='red', linestyle='--', alpha=0.7)
axes[1, 0].set_xlabel('Количество деревьев (n_estimators)', fontsize=12)
axes[1, 0].set_ylabel('ROC-AUC (тестовая выборка)', fontsize=12)
axes[1, 0].set_title('Качество на тестовой выборке', fontsize=14)
axes[1, 0].grid(True, alpha=0.3)

# График 4: Относительное улучшение ROC-AUC
axes[1, 1].plot(trees_df['n_estimators'][1:], 
                np.diff(trees_df['CV ROC-AUC mean']) / trees_df['CV ROC-AUC mean'].values[:-1] * 100,
                'd-', linewidth=2, markersize=8, color='green')
axes[1, 1].axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, label='Порог 0.1%')
axes[1, 1].set_xlabel('Количество деревьев (n_estimators)', fontsize=12)
axes[1, 1].set_ylabel('Относительное улучшение, %', fontsize=12)
axes[1, 1].set_title('Относительное улучшение ROC-AUC', fontsize=14)
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Важность признаков для оптимальной модели RF
print("\n📊 ВАЖНОСТЬ ПРИЗНАКОВ В ОПТИМАЛЬНОЙ МОДЕЛИ RANDOM FOREST:")

# Создание и обучение оптимальной модели RF
optimal_rf = RandomForestClassifier(
    n_estimators=optimal_trees,
    max_depth=optimal_depth if optimal_depth != "Без огр." else None,
    max_features=optimal_feat if optimal_feat not in ['sqrt', 'log2', 'all'] 
    else (optimal_feat if optimal_feat == 'sqrt' else 
          'log2' if optimal_feat == 'log2' else None),
    random_state=42,
    n_jobs=-1
)

optimal_rf.fit(X_train, y_train)

# Получение важности признаков
feature_importances = optimal_rf.feature_importances_
importance_df = pd.DataFrame({
    'Признак': feature_names,
    'Важность': feature_importances
}).sort_values('Важность', ascending=False)

print("\nТоп-5 самых важных признаков:")
print(importance_df.head().round(4).to_string(index=False))

# Визуализация важности признаков
plt.figure(figsize=(12, 8))
bars = plt.barh(importance_df['Признак'], importance_df['Важность'], color='teal')
plt.xlabel('Важность признака', fontsize=14)
plt.title(f'Важность признаков в Random Forest ({optimal_trees} деревьев)', fontsize=16)
plt.gca().invert_yaxis()

# Добавление значений на столбцы
for bar in bars:
    width = bar.get_width()
    plt.text(width + 0.001, bar.get_y() + bar.get_height()/2,
             f'{width:.3f}', ha='left', va='center', fontsize=10)

plt.tight_layout()
plt.show()

- - - - - - - - -
**Задание №2:**

Решить задачу классификации с использованием XGBoost. 

Исследовать время обучения, качество полученных результатов. Сравнить с данными полученными в п.1 и сделать выводы (в работе). 

Гиперпараметры модели подбираются в этой лабораторной вручную. Цель – попытаться найти лучшие значения и почувствовать рабочие диапазоны параметров.

In [ ]:
# доделать!!!